<a href="https://colab.research.google.com/github/Shaurya6355/Algo-Trading/blob/main/Stochastic_oscillator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install yfinance

In [17]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 18.1 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=88a6063cd6f73cd4dab9db7cb60869aa705e5ae928080dc2f9cae671bdac13ac
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [18]:
import pandas_ta as ta

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
tsla = yf.Ticker("TSLA")
tsla=tsla.history(period='6mo')[['Open', 'High', 'Low' ,'Close']]

In [15]:
k_period = 14
d_period = 3

In [16]:
tsla['n_high']=tsla['High'].rolling(14).max()
tsla['n_low'] = tsla['Low'].rolling(k_period).min()
tsla['%K'] = (tsla['Close'] - tsla['n_low']) * 100 / (tsla['n_high'] - tsla['n_low'])
tsla['%D'] = tsla['%K'].rolling(d_period).mean()

In [14]:
print(tsla['n_high'].head(20))

Date
2022-01-20            NaN
2022-01-21            NaN
2022-01-24            NaN
2022-01-25            NaN
2022-01-26            NaN
2022-01-27            NaN
2022-01-28            NaN
2022-01-31            NaN
2022-02-01            NaN
2022-02-02            NaN
2022-02-03            NaN
2022-02-04            NaN
2022-02-07            NaN
2022-02-08    1041.660034
2022-02-09    1004.549988
2022-02-10     987.690002
2022-02-11     987.690002
2022-02-14     987.690002
2022-02-15     947.770020
2022-02-16     947.770020
Name: n_high, dtype: float64


In [20]:
tsla.ta.stoch(high='high', low='low', k=14, d=3, append=True)

,STOCHk_14_3_3,STOCHd_14_3_3
Date,,
2022-02-08,NaN,NaN
2022-02-09,NaN,NaN
2022-02-10,58.482131,NaN
2022-02-11,52.707669,NaN
2022-02-14,45.019075,52.069625
...,...,...
2022-07-14,51.462847,53.204250
2022-07-15,57.471661,52.329294
2022-07-18,60.483232,56.472580


In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Avoid case-sensitive issues for accessing data.
# Optional if using pandas_ta
tsla.columns = [x.lower() for x in tsla.columns]
# Create our primary chart
# the rows/cols arguments tell plotly we want two figures
fig = make_subplots(rows=2, cols=1)  
# Create our Candlestick chart with an overlaid price line
fig.append_trace(
    go.Candlestick(
        x=tsla.index,
        open=tsla['open'],
        high=tsla['high'],
        low=tsla['low'],
        close=tsla['close'],
        increasing_line_color='#ff9900',
        decreasing_line_color='black',
        showlegend=False
    ), row=1, col=1  # <------------ upper chart
)
# price Line
fig.append_trace(
    go.Scatter(
        x=tsla.index,
        y=tsla['open'],
        line=dict(color='#ff9900', width=1),
        name='open',
    ), row=1, col=1  # <------------ upper chart
)
# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=tsla.index,
        y=tsla['stochk_14_3_3'],
        line=dict(color='#ff9900', width=2),
        name='fast',
    ), row=2, col=1  #  <------------ lower chart
)
# Slow signal (%d)
fig.append_trace(
    go.Scatter(
        x=tsla.index,
        y=tsla['stochd_14_3_3'],
        line=dict(color='#000000', width=2),
        name='slow'
    ), row=2, col=1 # <------------ lower chart
)
# Extend our y-axis a bit
fig.update_yaxes(range=[-10, 110], row=2, col=1)
# Add upper/lower bounds
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)
# Add overbought/oversold
fig.add_hline(y=20, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=80, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
fig.update_layout(layout)
# View our chart in the system default HTML viewer (Chrome, Firefox, etc.)
fig.show()